<a href="https://colab.research.google.com/github/anosharahim/deep-learning/blob/master/DL_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [11]:
## for preprocessing
import pandas as pd
import os
import numpy as np
import random 

from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split



In [7]:
#load data from gdrive
train_npz = np.load("/content/drive/MyDrive/Capstone/datasets/train.npz")
test_npz = np.load("/content/drive/MyDrive/Capstone/datasets/test.npz")

x_train = train_npz["arr_0"]
y_train = train_npz["arr_1"]
x_test = test_npz["arr_0"]
y_test = test_npz["arr_1"]

#add 2 more channels to grayscale image to imitate rgb
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)

#split test set into validation and test set 
x_val, x_test, y_val ,y_test = train_test_split(x_test,y_test, test_size=0.3)

print('Train -- ', x_train.shape, y_train.shape)
print('Val -- ', x_val.shape, y_val.shape)
print('Test -- ', x_test.shape, y_test.shape)

Train --  (28709, 48, 48, 3) (28709,)
Val --  (5024, 48, 48, 3) (5024,)
Test --  (2154, 48, 48, 3) (2154,)


In [18]:
from keras import layers, Model, Input

#change VGG input size from 224x224 to 48x48 to fit the input size
width_shape = 48
height_shape = 48
image_input = Input(shape=(width_shape, height_shape, 3))

vgg_model = VGG16(weights = "imagenet", include_top = False, input_tensor = image_input)
vgg_model.summary() 
#expects img size 224*224 with three channels 

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [24]:
import tensorflow.keras as K
adam = K.optimizers.Adam(learning_rate=0.0001)

#Freezing n Layers 
n_freeze = 10
for layer in vgg_model.layers[:n_freeze]:
  layer.trainable =False
#verify that the desired layers are frozen 
for i, layer in enumerate(vgg_model.layers):
  print(i,layer.name,'-',layer.trainable)


## Compile Model 
vgg_model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])


0 input_6 - False
1 block1_conv1 - False
2 block1_conv2 - False
3 block1_pool - False
4 block2_conv1 - False
5 block2_conv2 - False
6 block2_pool - False
7 block3_conv1 - False
8 block3_conv2 - False
9 block3_conv3 - False
10 block3_pool - True
11 block4_conv1 - True
12 block4_conv2 - True
13 block4_conv3 - True
14 block4_pool - True
15 block5_conv1 - True
16 block5_conv2 - True
17 block5_conv3 - True
18 block5_pool - True


In [26]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
#create path to checkpoint to save model weights
ckpt_path = os.path.join('/content/drive/MyDrive/Capstone/datasets', 'ckpt_vgg') 

#reduce learning rate when performance doesn't increase for 5 epochs
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 5,verbose = 1,factor = 0.1, min_lr = 1e-10)
#stop early if performance doesn't increase substantially for 10 epochs
#es = EarlyStopping(min_delta=.001, verbose=1, patience=15)

#create model checkpoint to save weights
check_point = K.callbacks.ModelCheckpoint(filepath=  ckpt_path,
                                              monitor="val_accuracy",
                                              mode="max",
                                              save_best_only=True,)

#Train the model using min--batch gradient descent
history = vgg_model.fit(x_train, y_train, batch_size=32, epochs=30, verbose=1,
                        validation_data=(x_val, y_val),
                        callbacks=[lrd, check_point])

vgg_model.save(ckpt_path)

Epoch 1/30
898/898 [==============================] - 100s 112ms/step - loss: 6.2493 - accuracy: 0.1389 - val_loss: 6.2383 - val_accuracy: 0.1290 - lr: 0.0010
Epoch 2/30
898/898 [==============================] - 96s 107ms/step - loss: 6.2383 - accuracy: 0.1392 - val_loss: 6.2383 - val_accuracy: 0.1290 - lr: 0.0010
Epoch 3/30
898/898 [==============================] - 91s 101ms/step - loss: 6.2383 - accuracy: 0.1392 - val_loss: 6.2383 - val_accuracy: 0.1290 - lr: 0.0010
Epoch 4/30
898/898 [==============================] - 91s 101ms/step - loss: 6.2383 - accuracy: 0.1392 - val_loss: 6.2383 - val_accuracy: 0.1290 - lr: 0.0010
Epoch 5/30
898/898 [==============================] - 96s 107ms/step - loss: 6.2383 - accuracy: 0.1391 - val_loss: 6.2383 - val_accuracy: 0.1290 - lr: 0.0010
Epoch 6/30
898/898 [==============================] - ETA: 0s - loss: 6.2383 - accuracy: 0.1392
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
898/898 [===========================

KeyboardInterrupt: ignored

# Project Overview 

# References


# Appendices 

## LO Applications
\#deeplearning

\#advanceddeeplearning

## HC Applications
\#algorithms

\#probability

\#variables

\#optimization 

\#distributions 

